# Lab 7: Turning Queries into Functions

## First, let's setup the database engine like we did last week

### Load DB credentials

In [1]:
import json

# TODO: make sure to download credentials from https://canvas.upenn.edu/files/89654914/download?download_frd=1
# save them to the base directory for this repo
with open("pg-credentials.json") as creds:
    creds = json.load(creds)

PASSWORD = creds["PASSWORD"]
HOST = creds["HOST"]
USERNAME = creds["USERNAME"]
DATABASE = creds["DATABASE"]
PORT = creds["PORT"]

### Create DB engine

In [2]:
from sqlalchemy import create_engine

engine = create_engine(f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

In [3]:
# make sure it works

rows = engine.execute("SELECT 'Hello' as hithere").fetchall()
rows

[('Hello',)]

In [4]:
rows[0]

('Hello',)

## Review

## Review from Lecture

* Query templates
* Putting a query template into a function for reusable code

## 0. Review: Query Templating

### Avoid SQL injection

![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

<https://imgs.xkcd.com/comics/exploits_of_a_mom.png>

### Let's see how SQLAlchemy templates our queries

**Valid Inputs**

In [22]:
from sqlalchemy.sql import text

q = text("SELECT name, totaldocks, docksavailable FROM indego_station_status LIMIT :num")

In [23]:
str(q.bindparams(num=10).compile(bind=engine, compile_kwargs={"literal_binds": True}))

'SELECT name, totaldocks, docksavailable FROM indego_station_status LIMIT 10'

In [24]:
engine.execute(q, num=5).fetchall()

[('11th & Poplar, John F. Street Community Center', 15, 11),
 ('Amtrak 30th Street Station', 21, 20),
 ('43rd & Chester, Clark Park', 19, 14),
 ('Front & Carpenter', 25, 22),
 ('Philadelphia Zoo', 12, 9)]

**Invalid Inputs**

In [ ]:
SELECT name, totaldocks, docksavailable FROM indego_station_status LIMIT 2; select * from andys_cookies;

In [25]:
str(q.bindparams(num='2; select * from andys_cookies;').compile(bind=engine, compile_kwargs={"literal_binds": True}))

"SELECT name, totaldocks, docksavailable FROM indego_station_status LIMIT '2; select * from andys_cookies;'"

In [26]:
engine.execute(q, num='2; select * from andys_cookies;').fetchall()

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type bigint: "2; select * from andys_cookies;"
LINE 1: ..., docksavailable FROM indego_station_status LIMIT '2; select...
                                                             ^

[SQL: SELECT name, totaldocks, docksavailable FROM indego_station_status LIMIT %(num)s]
[parameters: {'num': '2; select * from andys_cookies;'}]
(Background on this error at: http://sqlalche.me/e/13/9h9h)

Good news :) SQLAlchemy saved us!

### If we templated the string using Python string functions...

In [ ]:
q = f"""
SELECT a, b, c FROM (
  {subquery}
) as w
JOIN b
ON b.blah = c.a
"""

In [29]:
# DON'T DO THIS
qtext = "SELECT * FROM indego_station_status LIMIT {num}"

num = '2; SELECT * FROM andys_cookies;'
print(qtext.format(num=num))

SELECT * FROM indego_station_status LIMIT 2; SELECT * FROM andys_cookies;


**Uh oh**. Notice that another query was 'injected' into our templated query without proper quotes.

Let's execute it to see what happens...

In [28]:
engine.execute(qtext.format(num=num)).fetchall()

[('peanut butter', 10), ('pecan', 20), ('chocolate fudge', 5)]

My cookie table was hacked!

### Aside... creating a table from nothing

We'll discuss operations like this in the coming weeks, but I created that cookie table with this query:

```SQL
CREATE TABLE andys_cookies AS
SELECT cookie_type, quantity 
FROM (
	VALUES ('peanut butter', 10), 
	       ('pecan', 20),
	       ('chocolate fudge', 5)
) AS c(cookie_type, quantity)
```

## 1. More Templating Asking Questions

In [34]:
# NOTE: the dataset originally had capitals in the names, so we need to quote the column names here
def fetch_five_vacant_buildings():
    query = text("""
        SELECT "ADDRESS", "BLDG_DESC", "ZIPCODE", "BUILD_RANK"
        FROM vacant_buildings
        LIMIT 5
    """)
    return engine.execute(query).fetchall()

In [35]:
fetch_five_vacant_buildings()

[('3338 N WATER ST', 'ROW 2 STY MASONRY', '19134', 0.67),
 ('211 E LOUDON ST', 'SEMI DET 2 STY MASONRY', '19120', 0.67),
 ('2632 N 6TH ST', 'ROW 2 STY MASONRY', '19133', 0.5),
 ('5815 NORFOLK ST', 'ROW 2 STY MASONRY', '19143', 0.67),
 ('6427 N WOODSTOCK ST', 'ROW 2 STY MASONRY', '19138', 0.5)]

### 1.1 What are the five closest vacant buildings to Meyerson Hall?

Meyerson Hall has a lat/lng of `(39.952263,-75.1927827)`

In [36]:
def vacants_close_to_meyerson_hall(num_buildings=5):
    query = text("""
        SELECT "ADDRESS", geom
        FROM vacant_buildings
        ORDER BY ST_SetSRID(ST_MakePoint(-75.1927827, 39.952263), 4326) <-> geom
        LIMIT 5
    """)
    return engine.execute(query).fetchall()

In [37]:
vacants_close_to_meyerson_hall()

[('451 UNIVERSITY AVE', '0103000020E6100000010000000900000080A34FD2A4CC52C0375397F65EF943406BC1F025AACC52C0271CFD105AF94340C7067D49ADCC52C084B78F8B57F9434026E2E4F5B1CC52C0B2F ... (24 characters truncated) ... C52C0B80A55F552F943408CCB7E62B6CC52C00979AC4F55F94340CE6107FAB7CC52C03537C76B5CF943408571ECE6A8CC52C033CB52A969F9434080A34FD2A4CC52C0375397F65EF94340'),
 ('3631 WARREN ST', '0103000020E610000001000000050000001E0C7BC772CC52C0E1ADCFD0B0FA4340A91C026C75CC52C05139D8D7AAFA43401998E02176CC52C0FAFFB492ABFA43403619457973CC52C0070F868DB1FA43401E0C7BC772CC52C0E1ADCFD0B0FA4340'),
 ('201 N 36TH ST', '0103000020E610000001000000050000007F0423075CCC52C0359D4112BAFA4340EA76CAEB61CC52C005309EC5B9FA4340DA39D01162CC52C071B71C5DBBFA4340220B1B3A5CCC52C0E5BE9238BCFA43407F0423075CCC52C0359D4112BAFA4340'),
 ('3501-15 POWELTON AVE', '0103000020E610000001000000080000005187EF6155CC52C0798A155FF5FA4340E67E0AB054CC52C090615493E5FA43403147EFA965CC52C010503DDCE1FA43404AC05E2267CC52C0E6C40A8FF1FA4340D7B97D2467CC

### 1.2 What are the largest vacant buildings by zip code?

In [42]:
def vacants_close_to_meyerson_hall(zip_code):
    query = text("""
        SELECT "ADDRESS", "BLDG_DESC", "Shape__Area" as area_sq_m
        FROM vacant_buildings
        WHERE "ZIPCODE" = :zip_code
        ORDER BY "Shape__Area" DESC
        LIMIT 5
    """)
    return engine.execute(query, zip_code=zip_code).fetchall()

In [43]:
vacants_close_to_meyerson_hall("19104")

[('3501-15 POWELTON AVE', 'SCHOOL 2STY MASONRY', 8445.4921875),
 ('451 UNIVERSITY AVE', 'HEALTH FAC.CLINIC MAS+OTH', 6492.78125),
 ('4445 W GIRARD AVE', 'IND.WHSE MASONRY', 3541.0),
 ('609-11 N 43RD ST', 'APTS  5-50 UNTS MASONRY', 2472.19140625),
 ('905 N 41ST ST', 'APT.BOARDING HOME MASONRY', 2460.83984375)]

## 2. Give all vacant buildings in a neighborhood

### 2.1 Data

We have a neighborhood table

In [ ]:
resp = engine.execute("SELECT neighborhood_name, ST_AsText(geom)  FROM philadelphia_neighborhoods LIMIT 1").fetchall()
resp

### 2.2 Build a function that takes a neighborhood name and returns all vacant buildings in it

In [8]:
from sqlalchemy.sql import text

In [9]:
def vacant_buildings_by_neighborhood(name):
    query = text("""
    SELECT n.neighborhood_name, b."ADDRESS", b."BLDG_DESC"
    FROM philadelphia_neighborhoods as n
    JOIN vacant_buildings as b
    ON ST_Intersects(n.geom, b.geom)
    WHERE neighborhood_name = :name
    """)
    return engine.execute(query, name=name).fetchall()

In [11]:
vacant_buildings_by_neighborhood("University City")

[('University City', '4060 CHESTNUT ST', 'APTS  5-50 UNTS MASONRY'),
 ('University City', '4043 LUDLOW ST', 'ROW 3 STY MASONRY'),
 ('University City', '3631 WARREN ST', 'ROW 3 STY MASONRY'),
 ('University City', '451 UNIVERSITY AVE', 'HEALTH FAC.CLINIC MAS+OTH'),
 ('University City', '3922 SPRUCE ST', 'ROW 3 STY MASONRY')]

### 2.3 Let's Validate Inputs!

Validating inputs helps guide users if they make a mistake.

In [47]:
def is_valid_neighborhood_name(input_name):
    query = text("""
        SELECT neighborhood_name 
        FROM philadelphia_neighborhoods
        WHERE neighborhood_name = :input_name
    """)

    resp = engine.execute(query, input_name=input_name)
    if resp.rowcount > 0:
        return True
    return False

In [48]:
is_valid_neighborhood_name("Andy")

False

In [49]:
is_valid_neighborhood_name("Strawberry Mansion")

True

In [51]:
def get_vacant_buildings(neighborhood_name):
    if not is_valid_neighborhood_name(neighborhood_name):
        raise ValueError(f"'{neighborhood_name}' is not a valid neighborhood name")
    result = vacant_buildings_by_neighborhood(neighborhood_name)
    return result

In [52]:
get_vacant_buildings("Andy")

ValueError: 'Andy' is not a valid neighborhood name

### 2.4 But what are the valid names? Let's print them in the error message too.

Write a function to return the names of the neighborhoods

In [ ]:
def list_neighborhood_names():
    """Retrieve all neighborhood names, return as a list"""
    query = text("""
    --- put your query here
    """)
    # place your code here

Return should look like: 
```
['ACADEMY_GARDENS',
 'AIRPORT',
 'ALLEGHENY_WEST',
 'ANDORRA',
 'ASTON_WOODBRIDGE',
 'BARTRAM_VILLAGE',
 ...
```

### Now we can use the results of the list function to give users some options

In [ ]:
def get_vacant_buildings(neighborhood_name):
    if not is_valid_neighborhood_name(neighborhood_name):
        neighborhood_list = list_neighborhood_names()
        raise ValueError(f"'{neighborhood_name}' is not a valid neighborhood name. Choose one of {neighborhood_list}")
    pass

In [ ]:
get_vacant_buildings("Andy")

## 3. Fetching data from BigQuery

In [12]:
from google.cloud import bigquery
import geopandas as gpd
from shapely import wkt

# NOTE: you need to setup a service account (or use another auth method)
bqclient = bigquery.Client.from_service_account_json("MUSA-509-3337814ad805.json")

In [13]:
from shapely import wkt

query = f"""
SELECT (select value from unnest(all_tags) WHERE key = 'amenity') as amenity_type,
       COUNT(*) as num_amenities
  FROM `bigquery-public-data.geo_openstreetmap.planet_features`
 WHERE 'amenity' IN (SELECT key FROM UNNEST(all_tags))
 AND ST_INTERSECTSBOX(ST_Centroid(geometry), -75.280298,39.867005,-74.955831,40.137959)
GROUP BY 1
ORDER BY 2 DESC
"""
response = bqclient.query(query)

# print the rows
for row in response:
    print(row['amenity_type'].ljust(17), row['num_amenities'])

parking           3734
place_of_worship  1401
school            1054
restaurant        1033
fast_food         602
bench             554
social_facility   408
bank              273
cafe              244
fuel              244
parking_entrance  202
bar               192
fire_station      185
waste_basket      184
pharmacy          181
car_sharing       167
bicycle_parking   160
shelter           142
library           122
post_box          118
post_office       113
pub               88
marketplace       87
toilets           76
fountain          69
clinic            67
community_centre  65
theatre           64
atm               63
kindergarten      63
police            62
childcare         53
university        48
hospital          46
bicycle_rental    44
recycling         44
grave_yard        37
ice_cream         35
college           33
car_wash          33
doctors           33
dentist           32
car_rental        31
vending_machine   24
cinema            23
drinking_water    22
waste_dis

* [Parameterize queries](https://cloud.google.com/bigquery/docs/parameterized-queries) to avoid SQL Injection

BigQuery uses `@variable_name` notation for templating/parametrizing literals (strings, numbers, but not tables) in queries.

It makes use of the `QueryJobConfig` object in Python: <https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJobConfig.html>

In [19]:
def get_nearest_amenity(lng, lat, distance, amenity_type="cafe"):
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("poi_category", "STRING", amenity_type),
            bigquery.ScalarQueryParameter("lng", "FLOAT", lng),
            bigquery.ScalarQueryParameter("lat", "FLOAT", lat),
            bigquery.ScalarQueryParameter("distance", "FLOAT", distance)
        ]
    )
    query = f"""
        SELECT (select value from unnest(all_tags) WHERE key = 'name') as amenity_name, 
               (select value from unnest(all_tags) WHERE key = 'amenity') as amenity_type,
               (select value from unnest(all_tags) WHERE key = 'addr:street') as address,
               (select value from unnest(all_tags) WHERE key = 'phone') as phone_number,
               CAST(round(ST_Distance(ST_GeogPoint(@lng, @lat), ST_Centroid(geometry))) AS int64) as distance_away_meters,
               geometry
          FROM `bigquery-public-data.geo_openstreetmap.planet_features`
         WHERE ('amenity', @poi_category) IN (SELECT (key, value) FROM UNNEST(all_tags))
         and ST_DWithin(ST_GeogPoint(@lng, @lat), ST_Centroid(geometry), @distance)
         ORDER BY distance_away_meters ASC
    """
    response = bqclient.query(query, job_config=job_config)
    return response.result()

In [20]:
meyerson_lnglat = (-75.1927795, 39.9522139)
response = get_nearest_amenity(meyerson_lnglat[0], meyerson_lnglat[1], 1000, 'cafe')

In [21]:
for row in response:
    description = f"{row['amenity_name']} is {row['distance_away_meters']} meters away"
    if row['address'] is not None:
        description = description + f" on {row['address']}"
    print(description + '\n')

Starbucks is 98 meters away on Walnut Street

United By Blue is 102 meters away on Walnut Street

Avril 50 is 144 meters away on Sansom Street

Starbucks is 262 meters away

Kiwi Frozen Yougurt is 327 meters away

Saxby's is 377 meters away on Chestnut Street

Joe Coffee is 421 meters away on Chestnut

Starbucks is 510 meters away

Starbucks is 551 meters away

Starbucks is 564 meters away

Saxbys - Drexel is 604 meters away on North 34th Street

World Cafe Live is 650 meters away on Walnut Street

Starbucks is 678 meters away

Saxbys is 879 meters away on Locust Street

Green Line Cafe is 922 meters away on S 41st St

Metropolitan Bakery Café is 932 meters away on Walnut Street



In [ ]:
from cartoframes.viz import Layer

cafes = gpd.GeoDataFrame(response.to_dataframe(), geometry=[wkt.loads(row.geometry).centroid for row in response], crs="epsg:4326")

Layer(cafes)

### Add input validation

In [ ]:
query = f"""
SELECT DISTINCT (select value from unnest(all_tags) WHERE key = 'amenity') as amenity_type
  FROM `bigquery-public-data.geo_openstreetmap.planet_features`
 WHERE 'amenity' IN (SELECT key FROM UNNEST(all_tags))
 AND ST_INTERSECTSBOX(ST_Centroid(geometry), -75.280298,39.867005,-74.955831,40.137959)
"""
response = bqclient.query(query)

In [ ]:
poi_valid_set = set([row['amenity_type'] for row in response])
poi_valid_set

In [ ]:
def validate_poi_input(category):
    if category not in poi_valid_set:
        raise ValueError(f"`{category}` is not valid entry. Try one of {', '.join(poi_valid_set)}")

In [ ]:
validate_poi_input('hi')

In [ ]:
def get_nearest_cafes(lng, lat, distance, amenity_type="cafe"):
    validate_poi_input(amenity_type)
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("poi_category", "STRING", amenity_type),
            bigquery.ScalarQueryParameter("lng", "FLOAT", lng),
            bigquery.ScalarQueryParameter("lat", "FLOAT", lat),
            bigquery.ScalarQueryParameter("distance", "FLOAT", distance)
        ]
    )
    query = f"""
        SELECT (select value from unnest(all_tags) WHERE key = 'name') as amenity_name, 
               (select value from unnest(all_tags) WHERE key = 'amenity') as amenity_type,
               (select value from unnest(all_tags) WHERE key = 'addr:street') as address,
               (select value from unnest(all_tags) WHERE key = 'phone') as phone_number,
               CAST(round(ST_Distance(ST_GeogPoint(@lng, @lat), ST_Centroid(geometry))) AS int64) as distance_away_meters,
               geometry
          FROM `bigquery-public-data.geo_openstreetmap.planet_features`
         WHERE ('amenity', @poi_category) IN (SELECT (key, value) FROM UNNEST(all_tags))
         and ST_DWithin(ST_GeogPoint(@lng, @lat), ST_Centroid(geometry), @distance)
         ORDER BY distance_away_meters ASC
    """
    response = bqclient.query(query, job_config=job_config)
    return response

In [ ]:
get_nearest_cafes(meyerson_latlng[1], meyerson_latlng[0], 1000, 'bicycle_repair_station').to_dataframe()

## OpenStreetMap Editing

Are you interested in OSM for your project? There are many ways to get OSM data, including semi-yearly updates on BigQuery. There are daily extracts for regions of the world at [GeoFabrik](https://download.geofabrik.de/). The shapefiles can be big and hard to get down to the region of interest.

### Is OSM lacking in a region you want? Start adding your house, your parents house, etc.

<https://www.openstreetmap.org/#map=17/39.95484/-75.20505>